In [4]:
from collections import defaultdict
from typing import Dict, List, Tuple
import random
import copy

# Inputs
P_pref = {
    1: [2, 1, 3, 4, 4],
    2: [4, 1, 2, 3, 2],
    3: [1, 3, 2, 4, 1],
    4: [2, 3, 1, 4, 3],
}

A_pref = {
    1: [1, 3, 2, 4, 1],
    2: [3, 4, 1, 2, 3],
    3: [4, 2, 3, 1, 4],
    4: [3, 2, 1, 4, 2],
}

# DA
def deferred_acceptance(P: Dict[int, List[int]], A: Dict[int, List[int]]) -> Dict[int, int]:
    """Proposers-propose Gale–Shapley. Returns match_P: proposer -> acceptor (or None)."""
    A_rank = {a: {p: i for i, p in enumerate(order)} for a, order in A.items()}
    next_idx = {p: 0 for p in P}
    match_P = {p: None for p in P}
    match_A = {a: None for a in A}

    free = list(P.keys())
    while free:
        p = free.pop(0)

        # advance to next available acceptor on p's list
        while next_idx[p] < len(P[p]):
            a = P[p][next_idx[p]]
            next_idx[p] += 1
            if a in A_rank:   # only propose to acceptors that exist in A
                break
        else:
            # p has no one left to propose to
            continue

        current = match_A[a]
        if current is None:
            match_A[a] = p
            match_P[p] = a
        else:
            # acceptor chooses more preferred proposer
            if A_rank[a].get(p, 10**9) < A_rank[a].get(current, 10**9):
                match_A[a] = p
                match_P[p] = a
                match_P[current] = None
                free.append(current)
            else:
                free.append(p)
    return match_P

# Column poppping
def iterative_DA_with_column_popping(
    P_pref: Dict[int, List[int]],
    A_pref: Dict[int, List[int]],
    seed: int = 0
) -> Tuple[Dict[int, List[int]], Dict[int, List[int]]]:
    """
    For each 'column'/round:
      1) Run DA on current (P, A).
      2) For each proposer p, pick the round's choice with the following priority:
         a) DA match 'a' if exists, NOT previously chosen by p in earlier rounds, and NOT used this round.
         b) p's last preference from original P_pref, if it is NOT used this round and NOT previously chosen by p.
         c) otherwise: choose RANDOM from acceptors that are NOT used this round AND NOT previously chosen by p.
         (If that set is empty, relax to acceptors not used this round; if still empty, choose any acceptor.)
      3) Record choices, enforcing per-round uniqueness.
      4) Pop the first column (first item of every row) off both P and A.
    """
    random.seed(seed)
    P = copy.deepcopy(P_pref)
    A = copy.deepcopy(A_pref)

    rounds = max((len(v) for v in P_pref.values()), default=0)

    after_rows_proposers: Dict[int, List[int]] = {p: [] for p in P_pref}
    after_rows_acceptors: Dict[int, List[int]] = {a: [] for a in A_pref}

    # Track, per proposer, which acceptors have been chosen in prior rounds
    picked_by_p = {p: set() for p in P_pref}
    all_acceptors = set(A_pref.keys())

    for _ in range(rounds):
        match = deferred_acceptance(P, A)  # proposer -> acceptor (or None)
        used_this_round = set()            # enforce per-column uniqueness

        for p in sorted(P_pref.keys()):
            a = match.get(p)  # DA suggestion for this round (could be None)

            # helper: try to pick a candidate that is allowed and unused
            def choose_candidate():
                # 1) try DA match
                if (a is not None) and (a not in picked_by_p[p]) and (a not in used_this_round):
                    return a
                # 2) try last preference from original
                if P_pref[p]:
                    last_pref = P_pref[p][-1]
                    if (last_pref not in picked_by_p[p]) and (last_pref not in used_this_round):
                        return last_pref
                # 3) pick random from remaining unchosen (this round) and not used before by this proposer
                remaining = list(all_acceptors - used_this_round - picked_by_p[p])
                if remaining:
                    return random.choice(sorted(remaining))  # sorted for deterministic order before RNG
                # 3b) relax: anything unused this round
                remaining = list(all_acceptors - used_this_round)
                if remaining:
                    return random.choice(sorted(remaining))
                # 3c) final fallback: truly any
                return random.choice(sorted(all_acceptors))

            a_choice = choose_candidate()

            # record selections
            after_rows_proposers[p].append(a_choice)
            picked_by_p[p].add(a_choice)
            used_this_round.add(a_choice)
            after_rows_acceptors[a_choice].append(p)

        # pop first column off both tables
        for p in P:
            if P[p]:
                P[p].pop(0)
        for a in A:
            if A[a]:
                A[a].pop(0)

    return after_rows_proposers, after_rows_acceptors


# Running the function and then printing out the results
after_rows_proposers, after_rows_acceptors = iterative_DA_with_column_popping(P_pref, A_pref, seed=0)
print("after_rows_proposers =", after_rows_proposers)
print("after_rows_acceptors =", after_rows_acceptors)


after_rows_proposers = {1: [3, 4, 2, 1, 4], 2: [4, 1, 3, 2, 2], 3: [1, 3, 4, 4, 3], 4: [2, 2, 1, 3, 1]}
after_rows_acceptors = {1: [3, 2, 4, 1, 4], 2: [4, 4, 1, 2, 2], 3: [1, 3, 2, 4, 3], 4: [2, 1, 3, 3, 1]}
